В данном мини-проекте проведем ABC-анализ, XYZ-анализ и анализ динамики продаж товаров аптечной сети.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

Настроим максимальное значение отображаемых колонок, а также формат значений с плавающей точкой. Загрузим датафрейм.

In [2]:
pd.set_option('display.max_columns', 40)
pd.set_option('display.float_format', '{:.2f}'.format)

In [3]:
df = pd.read_csv(r'C:\Users\yarus\Downloads\Проект\Проект\data\data.csv', encoding='1251')

In [4]:
df.head()

,DR_Dat,DR_Tim,DR_NChk,DR_NDoc,DR_Apt,DR_Kkm,DR_TDoc,DR_TPay,DR_CDrugs,DR_NDrugs,DR_Suppl,DR_Prod,DR_Kol,DR_CZak,DR_CRoz,DR_SDisc,DR_CDisc,DR_BCDisc,DR_TabEmpl,DR_VZak,DR_Pos
0,2022-08-11,10:15:35,2173,2004598,2,22577,Розничная реализация,18,45399,ЦИПРОЛЕТ 3МГ/МЛ. 5МЛ. №1 ГЛ.КАПЛИ ФЛ./КАП. /Д-...,Катрен г.Химки,Д-р Редди с Лабораторис Лтд / Dr.REDDY's,1.00,41.08,51.00,12.00,925.00,200000000492.00,205,1,1.00
1,2022-08-11,10:27:46,2174,2004598,2,22577,Розничная реализация,15,261519,ПЕРЕКИСЬ ВОДОРОДА 3% 100МЛ. №40 Р-Р ФЛ.,Катрен г.Химки,ФЛОРА КАВКАЗА ОАО,1.00,18.61,31.00,3.00,9.00,200010010204.00,205,1,1.00
2,2022-08-11,10:27:46,2174,2004598,2,22577,Розничная реализация,15,460864,СОФЬЯ ГЕЛЬ Д/НОГ ВЕНОТОНИЗ. ТРОКСЕРУТИН ФОРТЕ ...,Катрен г.Химки,КОРОЛЕВФАРМ ООО,1.00,132.69,209.00,20.00,9.00,200010010204.00,205,1,2.00
3,2022-08-11,10:27:46,2174,2004598,2,22577,Розничная реализация,15,172823,СОФЬЯ ГХК КРЕМ Д/ТЕЛА ХОНДРОИТИН+ГЛЮКОЗАМИН 12...,Катрен г.Химки,КОРОЛЕВФАРМ ООО,1.00,133.65,210.00,21.00,9.00,200010010204.00,205,1,3.00
4,2022-08-11,10:33:56,2175,2004598,2,22577,Розничная реализация,18,79056,ГАЛВУС 50МГ. №28 ТАБ. /НОВАРТИС/,Катрен г.Химки,Новартис Фарма АГ,1.00,709.95,787.00,49.00,925.00,200000000492.00,205,1,1.00


## ABC-анализ

Сгруппируем по наименованию товара, и посчитаем его суммарное количество продаж. 

In [5]:
groupped_df = df.groupby('DR_NDrugs', as_index=False).agg({'DR_Kol': sum})

In [6]:
groupped_df.head()

,DR_NDrugs,DR_Kol
0,7 СЕМЬ ДНЕЙ МАСКА Д/ЛИЦА СУББОТА РОМАНТ. ПЕРЕД...,1.00
1,"9 МЕСЯЦЕВ ФОЛИЕВАЯ К-ТА 0,4МГ. №30 ТАБ. П/П/О ...",1.00
2,911-ГЕЛЬ-БАЛЬЗАМ Д/СУСТАВОВ ОКОПНИК 100МЛ. ТУБА,2.00
3,911-ГЕЛЬ-БАЛЬЗАМ Д/СУСТАВОВ САБЕЛЬНИК 100МЛ.,2.00
4,911-ГЕЛЬ-БАЛЬЗАМ Д/ТЕЛА БИШОФИТ П/БОЛИ В СУСТА...,1.00


In [7]:
groupped_df['DR_Kol'].max()

126.0

Посмотрим на доли в общем объеме проданного товара.

In [8]:
groupped_df = (groupped_df['DR_Kol'] / sum(groupped_df['DR_Kol'])).sort_values(ascending=False)

In [9]:
groupped_df.head()

1191   0.03
1769   0.01
1077   0.01
842    0.01
1426   0.01
Name: DR_Kol, dtype: float64

Посчитаем доли накопительно, чтобы выделить 80 % и 95 % срезы по объему проданных товаров.

In [10]:
groupped_df = groupped_df.cumsum().reset_index()

In [11]:
groupped_df.head()

,index,DR_Kol
0,1191,0.03
1,1769,0.04
2,1077,0.05
3,842,0.06
4,1426,0.07


Разделим товары на три группы по объему продаж.

In [12]:
groupped_df['abc'] = np.where(groupped_df['DR_Kol'] < 0.8, 'A', np.where(groupped_df['DR_Kol'] < 0.95, 'B', 'C'))

In [13]:
groupped_df[groupped_df['abc'] == 'B'].head()

,index,DR_Kol,abc
938,1817,0.80,B
939,1814,0.80,B
940,1812,0.80,B
941,1811,0.80,B
942,1809,0.80,B


Сгруппируем по наименованию товара, и посчитаем его суммарное количество продаж и суммарную стоимость продаж.

In [14]:
groupped_df = df.groupby('DR_NDrugs', as_index=False).agg({'DR_Kol': sum, 'DR_CRoz': sum})

In [15]:
groupped_df.head()

,DR_NDrugs,DR_Kol,DR_CRoz
0,7 СЕМЬ ДНЕЙ МАСКА Д/ЛИЦА СУББОТА РОМАНТ. ПЕРЕД...,1.00,93.00
1,"9 МЕСЯЦЕВ ФОЛИЕВАЯ К-ТА 0,4МГ. №30 ТАБ. П/П/О ...",1.00,144.00
2,911-ГЕЛЬ-БАЛЬЗАМ Д/СУСТАВОВ ОКОПНИК 100МЛ. ТУБА,2.00,216.00
3,911-ГЕЛЬ-БАЛЬЗАМ Д/СУСТАВОВ САБЕЛЬНИК 100МЛ.,2.00,208.00
4,911-ГЕЛЬ-БАЛЬЗАМ Д/ТЕЛА БИШОФИТ П/БОЛИ В СУСТА...,1.00,100.00


Посмотрим на доли в общем количестве товаров.

In [16]:
groupped_df['rel_kol'] = groupped_df['DR_Kol'] / sum(groupped_df['DR_Kol'])

In [17]:
groupped_df.head()

,DR_NDrugs,DR_Kol,DR_CRoz,rel_kol
0,7 СЕМЬ ДНЕЙ МАСКА Д/ЛИЦА СУББОТА РОМАНТ. ПЕРЕД...,1.00,93.00,0.00
1,"9 МЕСЯЦЕВ ФОЛИЕВАЯ К-ТА 0,4МГ. №30 ТАБ. П/П/О ...",1.00,144.00,0.00
2,911-ГЕЛЬ-БАЛЬЗАМ Д/СУСТАВОВ ОКОПНИК 100МЛ. ТУБА,2.00,216.00,0.00
3,911-ГЕЛЬ-БАЛЬЗАМ Д/СУСТАВОВ САБЕЛЬНИК 100МЛ.,2.00,208.00,0.00
4,911-ГЕЛЬ-БАЛЬЗАМ Д/ТЕЛА БИШОФИТ П/БОЛИ В СУСТА...,1.00,100.00,0.00


In [18]:
groupped_df = groupped_df.sort_values('rel_kol', ascending=False)

In [19]:
groupped_df.head()

,DR_NDrugs,DR_Kol,DR_CRoz,rel_kol
1191,ПАКЕТ,126.00,248.00,0.03
1769,"ЦЕФТРИАКСОН 1Г. №1 ПОР. Д/Р-РА Д/В/В,В/М ФЛ. /...",60.00,2280.00,0.01
1077,"НАФТИЗИН 0,1% 15МЛ. НАЗАЛ.КАПЛИ ФЛ./КАП. /ЛЕККО/",48.00,552.00,0.01
842,"ЛЕЙКОПЛАСТЫРЬ БАКТЕР. 2,5Х7,2 №1 /ВЕРОФАРМ/",40.00,28.00,0.01
1426,"СНУП 0,1% 90МКГ/ДОЗА 15МЛ. НАЗАЛ.СПРЕЙ ФЛ. /ШТ...",35.00,4215.00,0.01


Посчитаем доли накопительно.

In [20]:
groupped_df['cumsum_kol'] = groupped_df['rel_kol'].cumsum()

In [21]:
groupped_df.head()

,DR_NDrugs,DR_Kol,DR_CRoz,rel_kol,cumsum_kol
1191,ПАКЕТ,126.00,248.00,0.03,0.03
1769,"ЦЕФТРИАКСОН 1Г. №1 ПОР. Д/Р-РА Д/В/В,В/М ФЛ. /...",60.00,2280.00,0.01,0.04
1077,"НАФТИЗИН 0,1% 15МЛ. НАЗАЛ.КАПЛИ ФЛ./КАП. /ЛЕККО/",48.00,552.00,0.01,0.05
842,"ЛЕЙКОПЛАСТЫРЬ БАКТЕР. 2,5Х7,2 №1 /ВЕРОФАРМ/",40.00,28.00,0.01,0.06
1426,"СНУП 0,1% 90МКГ/ДОЗА 15МЛ. НАЗАЛ.СПРЕЙ ФЛ. /ШТ...",35.00,4215.00,0.01,0.07


Разделим товары на три группы по объему продаж, используя библиотеку numpy.

In [22]:
groupped_df['abc_kol'] = np.where(groupped_df['cumsum_kol'] < 0.8, 'A', np.where(groupped_df['cumsum_kol'] < 0.95, 'B', 'C'))

In [23]:
groupped_df

,DR_NDrugs,DR_Kol,DR_CRoz,rel_kol,cumsum_kol,abc_kol
1191,ПАКЕТ,126.00,248.00,0.03,0.03,A
1769,"ЦЕФТРИАКСОН 1Г. №1 ПОР. Д/Р-РА Д/В/В,В/М ФЛ. /...",60.00,2280.00,0.01,0.04,A
1077,"НАФТИЗИН 0,1% 15МЛ. НАЗАЛ.КАПЛИ ФЛ./КАП. /ЛЕККО/",48.00,552.00,0.01,0.05,A
842,"ЛЕЙКОПЛАСТЫРЬ БАКТЕР. 2,5Х7,2 №1 /ВЕРОФАРМ/",40.00,28.00,0.01,0.06,A
1426,"СНУП 0,1% 90МКГ/ДОЗА 15МЛ. НАЗАЛ.СПРЕЙ ФЛ. /ШТ...",35.00,4215.00,0.01,0.07,A
...,...,...,...,...,...,...
1067,"НАТРИЯ ХЛОРИД 0,9% 400МЛ. №16 Р-Р Д/ИНФ. КОНТ....",0.06,669.00,0.00,1.00,C
1513,"СФМ ШПРИЦ 2МЛ. 3-Х КОМП. 0,63Х32ММ 23G №100 [SFM]",0.05,918.00,0.00,1.00,C
490,ДЕРМАГРИП ХАЙ РИСК ПЕРЧАТКИ ЛАТ. СМОТР. Н/СТЕР...,0.04,2000.00,0.00,1.00,C
1516,"СФМ ШПРИЦ 50МЛ. 3-Х КОМП. 1,2X40ММ 18G №25 [SFM]",0.04,1324.00,0.00,1.00,C


Промаркеруем товары также на три группы, но уже по выручке от товара.

In [24]:
groupped_df['rel_roz'] = groupped_df['DR_CRoz'] / sum(groupped_df['DR_CRoz'])
groupped_df = groupped_df.sort_values('rel_roz', ascending=False)
groupped_df['cumsum_roz'] = groupped_df['rel_roz'].cumsum()
groupped_df['abc_roz'] = np.where(groupped_df['cumsum_roz'] < 0.8, 'A', np.where(groupped_df['cumsum_roz'] < 0.95, 'B', 'C'))

In [25]:
groupped_df

,DR_NDrugs,DR_Kol,DR_CRoz,rel_kol,cumsum_kol,abc_kol,rel_roz,cumsum_roz,abc_roz
1553,"ТЕРАФЛЮ ЛИМОН ОТ ГРИППА И ПРОСТУДЫ 22,1Г. №14 ...",7.00,15638.00,0.00,0.36,A,0.01,0.01,A
1097,НИМЕСИЛ 100МГ. 2Г. №30 ГРАН. Д/СУСП. Д/ПРИЕМА ...,2.80,14445.00,0.00,0.61,A,0.01,0.02,A
619,ИНГАВИРИН 90МГ. №10 КАПС. /ВАЛЕНТА/,18.00,13750.00,0.00,0.19,A,0.01,0.03,A
1724,ХАРТМАНН БРАНОЛИНД H ПОВЯЗКА СТЕР. 10Х20СМ. №3...,0.17,12908.00,0.00,1.00,C,0.01,0.04,A
1825,"ЭЛИКВИС 2,5МГ. №60 ТАБ. П/П/О /ПФАЙЗЕР/БРИСТОЛ...",5.00,12731.00,0.00,0.43,A,0.01,0.05,A
...,...,...,...,...,...,...,...,...,...
768,КОНТЕЙНЕР Д/СБОРА БИОМАТЕРИАЛА 60МЛ. +ШПАТЕЛЬ ...,1.00,10.00,0.00,1.00,C,0.00,1.00,C
165,АСКОРБИНОВАЯ К-ТА 25МГ. АПЕЛЬСИН №10 ТАБ. КРУТ...,1.00,10.00,0.00,0.92,B,0.00,1.00,C
169,АСКОРБИНОВАЯ К-ТА 25МГ. №10 ТАБ. КРУТКА САХ. /...,7.00,9.00,0.00,0.34,A,0.00,1.00,C
710,КЛИНСА БАХИЛЫ СТАНДАРТ №2 (1ПАРА),2.00,7.00,0.00,0.67,A,0.00,1.00,C


In [26]:
groupped_df[['DR_CRoz', 'DR_Kol', 'abc_kol', 'abc_roz']].reset_index().sort_values('DR_CRoz', ascending=False)

,index,DR_CRoz,DR_Kol,abc_kol,abc_roz
0,1553,15638.00,7.00,A,A
1,1097,14445.00,2.80,A,A
2,619,13750.00,18.00,A,A
3,1724,12908.00,0.17,C,A
4,1825,12731.00,5.00,A,A
...,...,...,...,...,...
1871,768,10.00,1.00,C,C
1872,165,10.00,1.00,B,C
1873,169,9.00,7.00,A,C
1874,710,7.00,2.00,A,C


Оставим только интересующие нас поля.

In [27]:
df1 = df[['DR_NDrugs', 'DR_Kol', 'DR_CRoz']]
df1.columns

Index(['DR_NDrugs', 'DR_Kol', 'DR_CRoz'], dtype='object')

In [28]:
df1.head()

,DR_NDrugs,DR_Kol,DR_CRoz
0,ЦИПРОЛЕТ 3МГ/МЛ. 5МЛ. №1 ГЛ.КАПЛИ ФЛ./КАП. /Д-...,1.00,51.00
1,ПЕРЕКИСЬ ВОДОРОДА 3% 100МЛ. №40 Р-Р ФЛ.,1.00,31.00
2,СОФЬЯ ГЕЛЬ Д/НОГ ВЕНОТОНИЗ. ТРОКСЕРУТИН ФОРТЕ ...,1.00,209.00
3,СОФЬЯ ГХК КРЕМ Д/ТЕЛА ХОНДРОИТИН+ГЛЮКОЗАМИН 12...,1.00,210.00
4,ГАЛВУС 50МГ. №28 ТАБ. /НОВАРТИС/,1.00,787.00


Напишем функцию для маркировки на АВС-группы по переданным полям.

In [29]:
def perform_abc(df, index):
    """
    На вход подается датафрейм пандас...
    """
    cols = list(df.columns)
    cols.remove(index)
    groupped_df = df.groupby(index).agg({col: sum for col in cols})
    for col in cols:
        groupped_df[f'rel_{col}'] = groupped_df[col] / sum(groupped_df[col])
        groupped_df = groupped_df.sort_values(f'rel_{col}', ascending=False)
        groupped_df[f'cumsum_{col}'] = groupped_df[f'rel_{col}'].cumsum()
        groupped_df[f'abc_{col}'] = np.where(groupped_df[f'cumsum_{col}'] < 0.8, 'A', np.where(groupped_df[f'cumsum_{col}'] < 0.95, 'B', 'C'))
    return groupped_df[cols + [f'abc_{col}' for col in cols]]
        

Проведем АВС-анализ по количесству продаж и выручке, использовав уже написанную функцию.

In [30]:
perform_abc(df1, index='DR_NDrugs')

,DR_Kol,DR_CRoz,abc_DR_Kol,abc_DR_CRoz
DR_NDrugs,,,,
"ТЕРАФЛЮ ЛИМОН ОТ ГРИППА И ПРОСТУДЫ 22,1Г. №14 ПОР. Д/Р-РА Д/ПРИЕМА ВНУТРЬ ПАК.",7.00,15638.00,A,A
НИМЕСИЛ 100МГ. 2Г. №30 ГРАН. Д/СУСП. Д/ПРИЕМА ВНУТРЬ ПАК. /ГУИДОТТИ/МЕНАРИНИ/,2.80,14445.00,A,A
ИНГАВИРИН 90МГ. №10 КАПС. /ВАЛЕНТА/,18.00,13750.00,A,A
ХАРТМАНН БРАНОЛИНД H ПОВЯЗКА СТЕР. 10Х20СМ. №30 ПЕРУАН.БАЛЬЗАМ /АРТ.4923462/ [BRANOLIND],0.17,12908.00,C,A
"ЭЛИКВИС 2,5МГ. №60 ТАБ. П/П/О /ПФАЙЗЕР/БРИСТОЛ-МАЙЕРС/",5.00,12731.00,A,A
...,...,...,...,...
КОНТЕЙНЕР Д/СБОРА БИОМАТЕРИАЛА 60МЛ. +ШПАТЕЛЬ СТЕР. И/У (БАНКА),1.00,10.00,C,C
АСКОРБИНОВАЯ К-ТА 25МГ. АПЕЛЬСИН №10 ТАБ. КРУТКА САХ. /АСКОПРОМ/,1.00,10.00,B,C
АСКОРБИНОВАЯ К-ТА 25МГ. №10 ТАБ. КРУТКА САХ. /АСКОПРОМ/,7.00,9.00,A,C


## Динамика продаж
Считаем таблицу с большим количеством данных (за 5 дней).

In [31]:
df = pd.read_csv(r"C:\Users\yarus\Downloads\Проект\Проект\data\data1.csv", encoding='1251')

In [32]:
df.head()

,DR_Dat,DR_Tim,DR_NChk,DR_NDoc,DR_Apt,DR_Kkm,DR_TDoc,DR_TPay,DR_CDrugs,DR_NDrugs,DR_Suppl,DR_Prod,DR_Kol,DR_CZak,DR_CRoz,DR_SDisc,DR_CDisc,DR_BCDisc,DR_TabEmpl,DR_VZak,DR_Pos
0,2022-08-01,08:06:18,1272,13002561,13,22589,Розничная реализация,18,144734,ГАСТАЛ №12 ТАБ. Д/РАСС.,Пульс,TEVA Pharvaceutical Industries Ltd,1.00,196.71,270.00,0.00,NaN,NaN,29,1,1.00
1,2022-08-01,08:38:53,1273,13002561,13,22589,Розничная реализация,15,69661,"ТОБРОПТ 0,3% 5МЛ. №1 ГЛ.КАПЛИ ФЛ./КАП. /РОМФАРМ/",Пульс,РОМФАРМ КОМПАНИ ( ROMPHARM ),1.00,106.21,127.00,6.00,9.00,200010004357.00,29,1,1.00
2,2022-08-01,08:55:38,1274,13002561,13,22589,Розничная реализация,18,190635,ЭЛИКВИС 5МГ. №60 ТАБ. П/П/О /ПФАЙЗЕР/БРИСТОЛ-М...,ГРАНД КАПИТАЛ СМОЛЕНСК ООО ФК,Пфайзер,1.00,2320.99,2563.00,76.00,9.00,200010018491.00,29,1,1.00
3,2022-08-01,09:00:40,1275,13002561,13,22589,Розничная реализация,18,276370,АРБИДОЛ МАКСИМУМ 200МГ. №10 КАПС. /ОТИСИФАРМ/Ф...,Пульс,ОТИСИФАРМ ПАО,1.00,445.39,541.00,0.00,NaN,NaN,29,1,1.00
4,2022-08-01,09:04:05,1276,13002561,13,22589,Розничная реализация,15,2303,"ЭНАМ 2,5МГ. №20 ТАБ. /Д-Р РЕДДИ/",Протек,Д-р Редди с Лабораторис Лтд / Dr.REDDY's,1.00,18.04,22.00,1.00,9.00,200010000734.00,29,1,5.00


In [33]:
df.DR_Dat.unique()

array(['2022-08-01', '2022-08-02', '2022-08-03', '2022-08-04',
       '2022-08-05'], dtype=object)

In [34]:
df.DR_Apt.unique()

array([13], dtype=int64)

Произведем рассчет выручки по каждой дате.

In [35]:
df = pd.read_csv(r"C:\Users\yarus\Downloads\Проект\Проект\data\data1.csv", encoding='1251')
df = df.groupby('DR_Dat').apply(lambda x: sum(x['DR_Kol']*x['DR_CRoz'] - x['DR_SDisc'])).reset_index()
df

,DR_Dat,0
0,2022-08-01,84681.52
1,2022-08-02,71389.37
2,2022-08-03,78050.82
3,2022-08-04,59187.36
4,2022-08-05,56458.81


Используем pandas-серию для именования рассчетного столбца.

In [36]:
df = pd.read_csv(r"C:\Users\yarus\Downloads\Проект\Проект\data\data1.csv", encoding='1251')
df = df.groupby('DR_Dat').apply(lambda x: pd.Series({'revenue': sum(x['DR_Kol']*x['DR_CRoz'] - x['DR_SDisc'])})).reset_index()
df

,DR_Dat,revenue
0,2022-08-01,84681.52
1,2022-08-02,71389.37
2,2022-08-03,78050.82
3,2022-08-04,59187.36
4,2022-08-05,56458.81


Либо можно было применить поверх переименование полей в датафрейме.

In [37]:
df.columns = ['DR_Dat', 'revenue']

In [38]:
df

,DR_Dat,revenue
0,2022-08-01,84681.52
1,2022-08-02,71389.37
2,2022-08-03,78050.82
3,2022-08-04,59187.36
4,2022-08-05,56458.81


Рассмотрим динамику выручки по дням, а именно процент изменения относительно прошедшего дня.

In [39]:
df['revenue_d'] = df['revenue'].rolling(2).apply(lambda x: (x.iloc[1] - x.iloc[0])/x.iloc[0])
df

,DR_Dat,revenue,revenue_d
0,2022-08-01,84681.52,NaN
1,2022-08-02,71389.37,-0.16
2,2022-08-03,78050.82,0.09
3,2022-08-04,59187.36,-0.24
4,2022-08-05,56458.81,-0.05


Либо можно было применить метод shift.

In [40]:
df['revenue_shifted'] = df['revenue'].shift(1)
df

,DR_Dat,revenue,revenue_d,revenue_shifted
0,2022-08-01,84681.52,NaN,NaN
1,2022-08-02,71389.37,-0.16,84681.52
2,2022-08-03,78050.82,0.09,71389.37
3,2022-08-04,59187.36,-0.24,78050.82
4,2022-08-05,56458.81,-0.05,59187.36


In [41]:
df['revenue_d2'] = (df['revenue'] - df['revenue_shifted']) / df['revenue_shifted']
df

,DR_Dat,revenue,revenue_d,revenue_shifted,revenue_d2
0,2022-08-01,84681.52,NaN,NaN,NaN
1,2022-08-02,71389.37,-0.16,84681.52,-0.16
2,2022-08-03,78050.82,0.09,71389.37,0.09
3,2022-08-04,59187.36,-0.24,78050.82,-0.24
4,2022-08-05,56458.81,-0.05,59187.36,-0.05


Также есть уже встроенный метод, считающий относительное изменение на заданное количество смещения.

In [42]:
df['revenue_d3'] = df['revenue'].pct_change(1)
df

,DR_Dat,revenue,revenue_d,revenue_shifted,revenue_d2,revenue_d3
0,2022-08-01,84681.52,NaN,NaN,NaN,NaN
1,2022-08-02,71389.37,-0.16,84681.52,-0.16,-0.16
2,2022-08-03,78050.82,0.09,71389.37,0.09,0.09
3,2022-08-04,59187.36,-0.24,78050.82,-0.24,-0.24
4,2022-08-05,56458.81,-0.05,59187.36,-0.05,-0.05


## XYZ-анализ

In [43]:
df = pd.read_csv(r"C:\Users\yarus\Downloads\Проект\Проект\data\data1.csv", encoding='1251')

In [44]:
df.head()

,DR_Dat,DR_Tim,DR_NChk,DR_NDoc,DR_Apt,DR_Kkm,DR_TDoc,DR_TPay,DR_CDrugs,DR_NDrugs,DR_Suppl,DR_Prod,DR_Kol,DR_CZak,DR_CRoz,DR_SDisc,DR_CDisc,DR_BCDisc,DR_TabEmpl,DR_VZak,DR_Pos
0,2022-08-01,08:06:18,1272,13002561,13,22589,Розничная реализация,18,144734,ГАСТАЛ №12 ТАБ. Д/РАСС.,Пульс,TEVA Pharvaceutical Industries Ltd,1.00,196.71,270.00,0.00,NaN,NaN,29,1,1.00
1,2022-08-01,08:38:53,1273,13002561,13,22589,Розничная реализация,15,69661,"ТОБРОПТ 0,3% 5МЛ. №1 ГЛ.КАПЛИ ФЛ./КАП. /РОМФАРМ/",Пульс,РОМФАРМ КОМПАНИ ( ROMPHARM ),1.00,106.21,127.00,6.00,9.00,200010004357.00,29,1,1.00
2,2022-08-01,08:55:38,1274,13002561,13,22589,Розничная реализация,18,190635,ЭЛИКВИС 5МГ. №60 ТАБ. П/П/О /ПФАЙЗЕР/БРИСТОЛ-М...,ГРАНД КАПИТАЛ СМОЛЕНСК ООО ФК,Пфайзер,1.00,2320.99,2563.00,76.00,9.00,200010018491.00,29,1,1.00
3,2022-08-01,09:00:40,1275,13002561,13,22589,Розничная реализация,18,276370,АРБИДОЛ МАКСИМУМ 200МГ. №10 КАПС. /ОТИСИФАРМ/Ф...,Пульс,ОТИСИФАРМ ПАО,1.00,445.39,541.00,0.00,NaN,NaN,29,1,1.00
4,2022-08-01,09:04:05,1276,13002561,13,22589,Розничная реализация,15,2303,"ЭНАМ 2,5МГ. №20 ТАБ. /Д-Р РЕДДИ/",Протек,Д-р Редди с Лабораторис Лтд / Dr.REDDY's,1.00,18.04,22.00,1.00,9.00,200010000734.00,29,1,5.00


Посмотрим на количество продаваемого товара в разрезе каждого дня.

In [45]:
df = df.groupby(['DR_Dat', 'DR_NDrugs'], as_index= False).agg({'DR_Kol': sum})
df

,DR_Dat,DR_NDrugs,DR_Kol
0,2022-08-01,"911-ВЕНОЛГОН ГЕЛЬ Д/НОГ ПРИ ТЯЖЕСТИ,БОЛИ,ОТЕКА...",1.00
1,2022-08-01,L-ТИРОКСИН 100МКГ. №100 ТАБ. /БЕРЛИН ХЕМИ/,1.00
2,2022-08-01,L-ТИРОКСИН 50МКГ. №50 ТАБ. /БЕРЛИН ХЕМИ/,1.00
3,2022-08-01,L-ТИРОКСИН 75МКГ. №100 ТАБ. /БЕРЛИН ХЕМИ/,2.00
4,2022-08-01,NF ВАТА ХИРУРГ. СТЕР. 100Г. /НЬЮФАРМ/,1.00
...,...,...,...
1084,2022-08-05,"ЭВО ПАНТЕНОЛ ПОМАДА ГИГИЕН. 2,8Г. [EVO] /АВАНТА/",1.00
1085,2022-08-05,ЭМОКСИПИН 10МГ/МЛ. 5МЛ. №1 ГЛ.КАПЛИ ФЛ. КРЫШ/КАП.,2.00
1086,2022-08-05,ЭСВИЦИН СР-ВО П/ОБЛЫСЕНИЯ ЛОСЬОН-ТОНИК 250МЛ. ...,1.00
1087,2022-08-05,ЭТОРИАКС 90МГ. №7 ТАБ. П/П/О,1.00


Отберем только те позиции, которые продавались больше одного дня.

In [46]:
df1 = df.groupby('DR_NDrugs').agg({'DR_Dat': 'count'}).reset_index()
xyz_names = list(df1[df1['DR_Dat'] > 1]['DR_NDrugs'])
xyz_names

['L-ТИРОКСИН 50МКГ. №50 ТАБ. /БЕРЛИН ХЕМИ/',
 'L-ТИРОКСИН 75МКГ. №100 ТАБ. /БЕРЛИН ХЕМИ/',
 'АЗАРГА 10МГ/МЛ.+5МГ/МЛ. 5МЛ. №1 ГЛ.КАПЛИ ФЛ./КАП.',
 'АМОКСИЦИЛЛИН 500МГ. №16 КАПС. /ХЕМОФАРМ/',
 'АРБИДОЛ МАКСИМУМ 200МГ. №10 КАПС. /ОТИСИФАРМ/ФАРМСТАНДАРТ/',
 'АСЕПТИКА САЛФЕТКА СПИРТОВАЯ 60Х100 №20',
 'АСКОРБИНКА ЛУНТИК ВИТ.С КЛУБНИКА №10 ТАБ. (30Г.) /ФАРМ-ПРО/ТИГОДА-ФАРМ/',
 'АСКОРБИНКА ЛУНТИК ВИТ.С №10 ТАБ. (30Г.) /ФАРМ-ПРО/ТИГОДА-ФАРМ/',
 'АСКОРБИНОВАЯ К-ТА 25МГ. ВИШНЯ №10 ТАБ. КРУТКА САХ. /АСКОПРОМ/',
 'АСКОРБИНОВАЯ К-ТА 25МГ. ЯБЛОКО №10 ТАБ. КРУТКА САХ. /АСКОПРОМ/',
 'АСПАРКАМ №60 ТАБ. /ФАРМАПОЛ-ВОЛГА/',
 'АСПЕРА СУПЕРЧИСТОТЕЛ 3МЛ. ФЛ.',
 'АТЕНОЛОЛ РЕНЕВАЛ 50МГ. №30 ТАБ. /ОБНОВЛЕНИЕ/',
 'АТОПИК КРЕМ-СТИК УСПОКАИВАЮЩИЙ Д/ДЕТ. 0+ 4,9МЛ. ПЕНАЛ [ATOPIC]',
 'АТОРВАСТАТИН-СЗ 20МГ. №60 ТАБ. П/П/О /СЕВЕРНАЯ ЗВЕЗДА/',
 'АФОБАЗОЛ 10МГ. №60 ТАБ.',
 'АЦЕТИЛСАЛИЦИЛОВАЯ К-ТА 500МГ. №20 ТАБ. /ФАРМСТАНДАРТ/',
 'АЦИКЛОВИР-БЕЛУПО 5% 10Г. №1 КРЕМ Д/НАРУЖ.ПРИМ. ТУБА /БЕЛУПО/',
 'БЕТАГИСТИН-СЗ 16МГ. №60 ТАБ

Оставим только эти товары для анализа и рассчитаем для каждого товара коэффициент вариативности.

In [47]:
df = df[df['DR_NDrugs'].isin(xyz_names)]
df = df.groupby('DR_NDrugs', as_index= False).apply(lambda x: x.std()/x.mean())
df

,DR_NDrugs,DR_Kol
0,L-ТИРОКСИН 50МКГ. №50 ТАБ. /БЕРЛИН ХЕМИ/,0.00
1,L-ТИРОКСИН 75МКГ. №100 ТАБ. /БЕРЛИН ХЕМИ/,0.47
2,АЗАРГА 10МГ/МЛ.+5МГ/МЛ. 5МЛ. №1 ГЛ.КАПЛИ ФЛ./КАП.,0.00
3,АМОКСИЦИЛЛИН 500МГ. №16 КАПС. /ХЕМОФАРМ/,0.47
4,АРБИДОЛ МАКСИМУМ 200МГ. №10 КАПС. /ОТИСИФАРМ/Ф...,0.43
...,...,...
177,"ЭВО ПАНТЕНОЛ ПОМАДА ГИГИЕН. 2,8Г. [EVO] /АВАНТА/",0.47
178,ЭЛИКВИС 5МГ. №60 ТАБ. П/П/О /ПФАЙЗЕР/БРИСТОЛ-М...,0.00
179,ЭМОКСИПИН 10МГ/МЛ. 5МЛ. №1 ГЛ.КАПЛИ ФЛ. КРЫШ/КАП.,0.47
180,ЭНАЛАПРИЛ 5МГ. №20 ТАБ. /ХЕМОФАРМ/,0.47


Промаркируем в зависимости от рассчитаннызх значений на XYZ-группы.

In [48]:
df['XYZ'] = np.where(df['DR_Kol'] < 0.1, 'X', np.where(df['DR_Kol'] < 0.25, 'Y', 'Z'))

In [49]:
df.head()

,DR_NDrugs,DR_Kol,XYZ
0,L-ТИРОКСИН 50МКГ. №50 ТАБ. /БЕРЛИН ХЕМИ/,0.00,X
1,L-ТИРОКСИН 75МКГ. №100 ТАБ. /БЕРЛИН ХЕМИ/,0.47,Z
2,АЗАРГА 10МГ/МЛ.+5МГ/МЛ. 5МЛ. №1 ГЛ.КАПЛИ ФЛ./КАП.,0.00,X
3,АМОКСИЦИЛЛИН 500МГ. №16 КАПС. /ХЕМОФАРМ/,0.47,Z
4,АРБИДОЛ МАКСИМУМ 200МГ. №10 КАПС. /ОТИСИФАРМ/Ф...,0.43,Z
